# Online Record Store Analysis

In this project, I will be using the Chinook database that can be found [here](https://github.com/lerocha/chinook-database).

This is a sample database for an online digital media store, including tables for artists, albums, media tracks, invoices and customers. I will use this database to answer some hypothetical business questions.

First, I need to connect to the database:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Overview of the Data

I will explore the tables in the database, and get an idea of the first 10 rows of one of the tables.

In [2]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql

SELECT *
  FROM invoice
 LIMIT 10;

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83
6,31,2017-01-10 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98
7,49,2017-01-12 00:00:00,Ordynacka 10,Warsaw,None,Poland,00-358,10.89
8,59,2017-01-13 00:00:00,"3,Raj Bhavan Road",Bangalore,None,India,560001,9.9
9,18,2017-01-18 00:00:00,627 Broadway,New York,NY,USA,10012-2612,8.91
10,31,2017-01-18 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98


## Selecting Albums to Purchase

The store would like to select 3 new albums to add to the store. There are 4 artists to choose from:

| Artist Name | Genre |
| --- | --- | 
| Regal	| Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

In order to inform this decision, I will figure out which genre sells the most tracks in the USA.

In [4]:
%%sql

WITH 
    usa_genres AS
    (
    SELECT i.billing_country country,
           g.name genre_name
      FROM invoice i
     INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
     INNER JOIN track t ON t.track_id = il.track_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     WHERE country = 'USA'
    )
    
SELECT genre_name genre,
       COUNT(genre_name) tracks_sold,
       CAST(COUNT(genre_name) AS Float) / (
                                           SELECT COUNT(genre_name)
                                             FROM usa_genres
                                            ) tracks_sold_pct
  FROM usa_genres
 GROUP BY genre_name
 ORDER BY tracks_sold DESC;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,tracks_sold_pct
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the highest selling genres in the USA, we should definitely choose the Punk album, and should probably go with the Blues album as well. For the third album it is a bit of a toss up between the Pop and the Hip Hop albums, as their genres only sold 22 and 20 tracks in the USA, respectively.

The most popular genre in the USA is by far Rock, so we should be on the lookout for rock albums to purchase instead of lower-selling genres like Pop and Rap.

## Analyzing Employee Sales Performance

Next I want to evaluate the performace of the sales support agents. I will calculate the total amount of sales attributed to each agent.

In [5]:
%%sql

WITH 
    agent_info AS
    (
    SELECT e.*, i.total
      FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
     INNER JOIN employee e ON e.employee_id = c.support_rep_id
     WHERE e.title = 'Sales Support Agent'
    )
    
SELECT first_name || ' ' || agent_info.last_name employee_name,
       birthdate,
       hire_date,
       ROUND(SUM(total), 2) total_sales
  FROM agent_info
 GROUP BY employee_id
 ORDER BY total_sales DESC;

 * sqlite:///chinook.db
Done.


employee_name,birthdate,hire_date,total_sales
Jane Peacock,1973-08-29 00:00:00,2017-04-01 00:00:00,1731.51
Margaret Park,1947-09-19 00:00:00,2017-05-03 00:00:00,1584.0
Steve Johnson,1965-03-03 00:00:00,2017-10-17 00:00:00,1393.92


There is a difference in the total sales for the 3 Sales Support Agents, but this could be explained by the date they were hired. If the agents are randomly assigned when a customer makes their first purchase, it would be surprising if their sales numbers were significantly different. It is likely that Jane Peacock has more sales simply because they have been working the job the longest.

## Analyzing Sales by Country

I would like to calculate sales data for customers from each different country. In particular, I will calculate the following for each country:
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Countries with only one customer will be grouped together as "Other"

In [6]:
%%sql

WITH 
    invoice_with_country AS
    (
    SELECT i.*, c.country
      FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
    ),
    
    country_counts AS
    (
    SELECT country,
           COUNT(country) num_orders,
           COUNT(DISTINCT(customer_id)) num_customers,
           SUM(total) total_sales,
           SUM(total) / COUNT(DISTINCT(customer_id)) sales_per_customer,
           SUM(total) / COUNT(country) avg_order_value
      FROM invoice_with_country
     GROUP BY country
    ),
    
    other_countries AS
    (
    SELECT 'Other' country,  
           SUM(num_orders) num_orders,
           SUM(num_customers) num_customers,
           SUM(total_sales) total_sales,
           SUM(total_sales) / SUM(num_customers) sales_per_customer,
           SUM(total_sales) / SUM(num_orders) avg_order_value
      FROM country_counts 
     WHERE num_customers = 1
    ),
    
    country_counts_other AS
    (
    SELECT *
      FROM country_counts
     WHERE num_customers > 1

     UNION

    SELECT *
      FROM other_countries
    )
    

SELECT country,
       num_customers,
       ROUND(total_sales, 2) total_sales,
       ROUND(sales_per_customer, 2) sales_per_customer,
       ROUND(avg_order_value, 2) avg_order_value
  FROM (
        SELECT *,        
                CASE
                    WHEN country = 'Other' THEN 1
                    ELSE 0
                END AS sort
        FROM country_counts_other
        )
 ORDER BY sort ASC, total_sales DESC;

 * sqlite:///chinook.db
Done.


country,num_customers,total_sales,sales_per_customer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


There is potentially an opportunity in countries like India, the United Kingdom, and the Czech Republic, which have higher than average total order values.  However, these countries only have 2 or 3 customers, which is too small a sample size to draw any meaningful conclusions.

## Albums vs. Individual Tracks

Customers are able to make purchases in two different way:
- purchase a whole album
- purchase a collection of one or more individual tracks

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

I want to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [7]:
%%sql 

WITH invoice_albums AS
    (
    SELECT i.invoice_id,
           al.album_id,
           title album_title
      FROM invoice i
     INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
     INNER JOIN track t ON t.track_id = il.track_id
     INNER JOIN album al ON al.album_id = t.album_id
    ),
    
    categorized_invoices AS
    (
    SELECT *,
        CASE
            WHEN (
                 SELECT track_id
                   FROM track t
                  WHERE album_id = ia.album_id

                 EXCEPT

                 SELECT track_id
                   FROM invoice_line il
                  WHERE invoice_id = ia.invoice_id
                 ) IS NULL
        
            AND
        
                 (
                 SELECT track_id
                   FROM invoice_line il
                  WHERE invoice_id = ia.invoice_id
                 
                 EXCEPT
             
                 SELECT track_id
                   FROM track t
                  WHERE album_id = ia.album_id
                 ) IS NULL
         
            THEN 'yes'
            ELSE 'no'
        END AS full_album
    FROM invoice_albums ia
    ),
    
    regrouped AS
    (
    SELECT *
      FROM categorized_invoices
     GROUP BY invoice_id
    )

SELECT full_album,
       COUNT(full_album) num_invoices,
       CAST(COUNT(full_album) AS Float) / (
                                           SELECT COUNT(*)
                                             FROM regrouped
                                           ) pct
  FROM regrouped r
 GROUP BY full_album;


 * sqlite:///chinook.db
Done.


full_album,num_invoices,pct
no,500,0.8143322475570033
yes,114,0.18566775244299674


Out of 614 total invoices, 81% of them were not full album purchases. It could be a good strategy to stop purcahsing full albums, since this would only impact 19% of orders. Depending on the cost of purchasing songs, this strategy could save money. 

More analysis is needed to find out if this is a worthwile strategy.

## Questions for Further Investigation

- Which artist is used in the most playlists?
- How many tracks have been purchased vs not purchased?
- Is the range of tracks in the store reflective of their sales popularity?
- Do protected vs non-protected media types have an effect on popularity?